In [69]:
# Google Colab related imports
import os
import importlib.util

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Google Colab")
   # Install the easydiffraction library if it is not installed (including charts extras)
   if importlib.util.find_spec("easydiffraction") is None:
       !pip install 'easydiffraction[charts] @ git+https://github.com/easyscience/easydiffractionlib.git@new_job_dev'
   # Download the data files to be read in notebook
   if not os.path.exists('hrpt.xye'): 
       !wget https://raw.githubusercontent.com/EasyScience/EasyDiffractionLib/refs/heads/new_job_dev/examples/Fitting_PD-CW_La0.5Ba0.5CoO3-HRPT@PSI/hrpt.xye

In [70]:
# Import the easydiffraction library
import easydiffraction as ed

## Job

In [71]:
# Create a job - the main object to store all the information
job = ed.Job()

## Model

In [72]:
# Create a phase object
phase = ed.Phase(name='lbco')

In [73]:
# Set space group
phase.space_group.name_hm_alt = 'P m -3 m'

In [74]:
# Set cell parameters
phase.cell.length_a = 3.88

In [75]:
# Add atoms
phase.atom_sites.append(label='La', 
                        type_symbol='La', 
                        fract_x=0, 
                        fract_y=0, 
                        fract_z=0, 
                        occupancy=0.5,
                        b_iso_or_equiv=0.4958)
phase.atom_sites.append(label='Ba', 
                        type_symbol='Ba', 
                        fract_x=0, 
                        fract_y=0, 
                        fract_z=0, 
                        occupancy=0.5,
                        b_iso_or_equiv=0.4943)
phase.atom_sites.append(label='Co', 
                        type_symbol='Co', 
                        fract_x=0.5, 
                        fract_y=0.5, 
                        fract_z=0.5, 
                        b_iso_or_equiv=0.2567)
phase.atom_sites.append(label='O', 
                        type_symbol='O', 
                        fract_x=0, 
                        fract_y=0.5, 
                        fract_z=0.5, 
                        b_iso_or_equiv=1.4041)

In [76]:
# Show phase info in CIF format
print(phase.cif)

data_lbco
_cell_length_a 3.88
_cell_length_b 3.00000000
_cell_length_c 3.00000000
_cell_angle_alpha 90.00000000
_cell_angle_beta 90.00000000
_cell_angle_gamma 90.00000000
_space_group_name_H-M_ref 'P m -3 m'

loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
_atom_site_occupancy
_atom_site_adp_type
_atom_site_B_iso_or_equiv
La La 0.00000000 0.00000000 0.00000000 0.5 Biso 0.4958
Ba Ba 0.00000000 0.00000000 0.00000000 0.5 Biso 0.4943
Co Co 0.5 0.5 0.5 1.00000000 Biso 0.2567
O O 0.00000000 0.5 0.5 1.00000000 Biso 1.4041


In [77]:
# Add phase to the job object
job.add_phase(phase=phase)
print(job.phases)

Collection of 1 phases: ['lbco']


## Experiment

In [78]:
# Load experimentally measured data from a file in XYE format
job.add_experiment_from_file('hrpt.xye')

In [79]:
pattern_params = job.pattern
experiment_params = job.parameters

In [80]:
job.show_experiment_chart()

# Analysis

In [81]:
# Display the analysis chart before setting initial parameter values
job.show_analysis_chart()

In [82]:
# Modify model-related parameters
phase.scale = 8.5
phase.cell.length_a = 3.88

# Modify experiment-related parameters
pattern_params.zero_shift = 0

experiment_params.wavelength = 1.494
experiment_params.resolution_u = 0.1
experiment_params.resolution_v = -0.1
experiment_params.resolution_w = 0.2
experiment_params.resolution_x = 0
experiment_params.resolution_y = 0

job.set_background([(10.0, 174.3),
                    (20.0, 159.8),
                    (30.0, 167.9),
                    (50.0, 166.1),
                    (70.0, 172.3),
                    (90.0, 171.1),
                    (110.0, 172.4),
                    (130.0, 182.5),
                    (150.0, 173.0),
                    (165.0, 171.1)])

In [83]:
# Display the analysis chart after modifying parameters
job.show_analysis_chart()

In [84]:
# Select the model-related parameters to be fitted
phase.scale.free = True
phase.cell.length_a.free = True

# Select the experiment-related parameters to be fitted
pattern_params.zero_shift.free = True

experiment_params.resolution_u.free = True
experiment_params.resolution_v.free = True
experiment_params.resolution_w.free = True
experiment_params.resolution_y.free = True

In [85]:
# Print parameters before fitting
job.print_free_parameters()

<Parameter 'length_a': 3.88+/-0 Å, bounds=[0:inf]>
<Parameter 'scale': 8.5+/-0, bounds=[0:inf]>
<Parameter 'resolution_u': 0.1+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.1+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.2+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_y': 0.0+/-0, bounds=[-inf:inf]>


In [86]:
# Start Least-Squares minimization to fit the selected parameters
job.fit(method='leastsq', tolerance=1e-4)

In [87]:
# Print parameters after fitting
job.print_free_parameters()

<Parameter 'length_a': 3.89088+/-0.00004 Å, bounds=[0:inf]>
<Parameter 'scale': 9.122+/-0.031, bounds=[0:inf]>
<Parameter 'resolution_u': 0.0821+/-0.0030, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.116+/-0.007, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.1213+/-0.0032, bounds=[-inf:inf]>
<Parameter 'resolution_y': 0.0824+/-0.0019, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.6227+/-0.0010, bounds=[-inf:inf]>


In [88]:
# Display the analysis chart after fitting
job.show_analysis_chart()